## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [3]:
import pandas as pd
import numpy as np
import cassandra
import csv
path:str=r"event_data.csv"


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [18]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

from cassandra.cluster import Cluster
try:
    cluster=Cluster(["127.0.0.1"])
    session=cluster.connect()
    if session:
        print("Session Was Created Successfuly")
except Exception as Er:
    print(Er)        


Session Was Created Successfuly


#### Create & Set Keyspace

In [19]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nosqldb 
    WITH REPLICATION ={'class' : 'SimpleStrategy','replication_factor' : 1}
""")
    session.set_keyspace('nosqldb')
    print("Session Keyspace Was Set Sucessfully") 
except Exception as Er:
    print(Er)        



Session Keyspace Was Set Sucessfully


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [6]:
df=pd.read_csv(path)
df.head(30)

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
0,Pavement,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",345,Mercy:The Laundromat,10
1,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",438,Horn Concerto No. 4 in E flat K495: II. Romanc...,53
2,Gary Allan,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",438,Nothing On But The Radio,53
3,Charttraxx Karaoke,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",438,Fireflies,53
4,The Libertines,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",389,The Good Old Days,29
5,Huey Lewis And The News,Jacqueline,F,2,Lynch,245.52444,paid,"Atlanta-Sandy Springs-Roswell, GA",389,Hip To Be Square,29
6,JOSEF LOCKE & ORCHESTRA,Jacqueline,F,3,Lynch,166.26893,paid,"Atlanta-Sandy Springs-Roswell, GA",389,How Can You Buy Killarney (1992 Digital Remaster),29
7,Train,Jacqueline,F,4,Lynch,216.76363,paid,"Atlanta-Sandy Springs-Roswell, GA",389,Hey_ Soul Sister,29
8,Blind Pilot,Jacqueline,F,5,Lynch,156.94322,paid,"Atlanta-Sandy Springs-Roswell, GA",389,I Buried a Bone,29
9,Anjulie,Jacqueline,F,6,Lynch,194.63791,paid,"Atlanta-Sandy Springs-Roswell, GA",389,Boom,29


In [20]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command





# Assigning the Columns name and Their DataTypes save Then in The Variable Called data
try:
   
    
    df=pd.read_csv(path)  
    data=""
    for i in range(len(df.columns)): 
        if df.dtypes.iloc[i] == "object":
            data += f" {df.columns[i]} text ,"   
        elif df.dtypes.iloc[i] == "int" :
            data += f" {df.columns[i]} int ,"
        elif df.dtypes.iloc[i] == "float" :
            data += f" {df.columns[i]} float ,"
        else:
            data += f" {df.columns[i]} text ,"
                               
except Exception as ex:
    print(ex)
            
    


table="CREATE TABLE IF NOT EXISTS nosqldb.music_Library "
table= table +f"({data} PRIMARY KEY ((session_number), item_in_session_number))"
try:
    session.execute(table)
    print("New Table was created ")
except Exception as ex:
    print(ex)   
    
    
                    

New Table was created 


### Let's insert our data into of table

In [21]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

text=""
for i in str(tuple(i for i in df.columns)):
    if i !="'":
        text+=i
        
text  
count=0


for index, Row in df.iterrows():      
        try:
            query = f"INSERT INTO  nosqldb.music_Library {text}"
   
        
            ## Task: Write the INSERT statements and assign it to the query variable
            query=query + " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            
            ## Task: Match the column in the csv file to the column in the INSERT statement.
            ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
            ## e.g., if you want to INSERT location from csv file into database you will use row[7]
            session.execute(query, tuple(Row))
            count+=1
        except Exception as ex:
            print(ex) 
print(f"Data Was Inserted Sucessfully. Number of data inserted :{count} ")   

Data Was Inserted Sucessfully. Number of data inserted :6820 


### Validate our Data Model using a SELECT

In [22]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

#1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4
query=""" SELECT artist_name,song_title,length from music_Library WHERE session_number = 338 AND item_in_session_number = 4
"""
try:
    row=session.execute(query)
    count=0
    for rows in row:
        count+=1
        print(f"{count}:",rows.artist_name," | ",rows.song_title," | ",rows.length) 
       
except Exception as ex:
    print(ex)    

1: Faithless  |  Music Matters (Mark Knight Dub)  |  495.30731201171875


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [23]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command    


# Assigning the Columns name and Their DataTypes save Then in The Variable Called data
try:
   
    
    df=pd.read_csv(path)  
    data=""
    for i in range(len(df.columns)): 
        if df.dtypes.iloc[i] == "object":
            data += f" {df.columns[i]} text ,"   
        elif df.dtypes.iloc[i] == "int" :
            data += f" {df.columns[i]} int ,"
        elif df.dtypes.iloc[i] == "float" :
            data += f" {df.columns[i]} float ,"
        else:
            data += f" {df.columns[i]} text ,"
                               
except Exception as ex:
    print(ex)
            
    


table="CREATE TABLE IF NOT EXISTS nosqldb.music_Library2 "
table= table +f"({data} PRIMARY KEY ((user_id, session_number), item_in_session_number))"
try:
    session.execute(table)
    print("New Table was created ")
except Exception as ex:
    print(ex)   
    
    
                    
               

New Table was created 


### Let's insert our data into of table

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
text=""
for i in str(tuple(i for i in df.columns)):
    if i !="'":
        text+=i
        
text  
count=0


for index, Row in df.iterrows():      
        try:
            query = f"INSERT INTO  nosqldb.music_Library2 {text}"
   
        
            ## Task: Write the INSERT statements and assign it to the query variable
            query=query + " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            
            ## Task: Match the column in the csv file to the column in the INSERT statement.
            ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
            ## e.g., if you want to INSERT location from csv file into database you will use row[7]
            session.execute(query, tuple(Row))
            count+=1
        except Exception as ex:
            print(ex) 
print(f"Data Was Inserted Sucessfully. Number of data inserted :{count} ")   

Data Was Inserted Sucessfully. Number of data inserted :6820 


### Validate our Data Model using a SELECT

In [25]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

   #2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182

try:
    row=session.execute("select artist_name,song_title ,(fname,lname) as name  from music_library2 WHERE user_id = 10 AND session_number = 182 ORDER BY item_in_session_number ")
    count=0
    for rows in row:
        count+=1
        print(f"{count}:",rows.name," | ",rows.artist_name," | ",rows.song_title," | ")
except Exception as es:
    print(es)  

1: ('Sylvie', 'Cruz')  |  Down To The Bone  |  Keep On Keepin' On  | 
2: ('Sylvie', 'Cruz')  |  Three Drives  |  Greece 2000  | 
3: ('Sylvie', 'Cruz')  |  Sebastien Tellier  |  Kilometer  | 
4: ('Sylvie', 'Cruz')  |  Lonnie Gordon  |  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  | 


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [26]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command  

# Assigning the Columns name and Their DataTypes save Then in The Variable Called data
try:
   
    df=pd.read_csv(path)  
    data=""
    for i in range(len(df.columns)): 
        if df.dtypes.iloc[i] == "object":
            data += f" {df.columns[i]} text ,"   
        elif df.dtypes.iloc[i] == "int" :
            data += f" {df.columns[i]} int ,"
        elif df.dtypes.iloc[i] == "float" :
            data += f" {df.columns[i]} float ,"
        else:
            data += f" {df.columns[i]} text ,"
                               
except Exception as ex:
    print(ex)
            
    


table="CREATE TABLE IF NOT EXISTS nosqldb.music_Library3 "
table= table +f"({data} PRIMARY KEY (song_title, user_id))"
try:
    session.execute(table)
    print("New Table was created ")
except Exception as ex:
    print(ex)   
                     

New Table was created 


### Let's insert our data into of table

In [27]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below

text=""
for i in str(tuple(i for i in df.columns)):
    if i !="'":
        text+=i
        
text  
count=0


for index, Row in df.iterrows():      
        try:
            query = f"INSERT INTO  nosqldb.music_Library2 {text}"
   
        
            ## Task: Write the INSERT statements and assign it to the query variable
            query=query + " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            
            ## Task: Match the column in the csv file to the column in the INSERT statement.
            ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
            ## e.g., if you want to INSERT location from csv file into database you will use row[7]
            session.execute(query, tuple(Row))
            count+=1
        except Exception as ex:
            print(ex) 
print(f"Data Was Inserted Sucessfully. Number of data inserted :{count} ")   

Data Was Inserted Sucessfully. Number of data inserted :6820 


### Validate our Data Model using a SELECT

In [28]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
#3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'

try:
    row=session.execute("select  (fname,lname) as name  from music_library2 WHERE song_title = 'All Hands Against His Own' ALLOW FILTERING")
    count=0
    for rows in row:
        count+=1
        print(f"{count}:",rows.name)
except Exception as es:
    print(es) 

1: ('Tegan', 'Levine')
2: ('Jacqueline', 'Lynch')
3: ('Sara', 'Johnson')


### Drop the tables before closing out the sessions

In [29]:
for i in ["music_Library","music_Library2","music_Library3"]:
    try:
        session.execute(f"DROP TABLE {i}")
        print(f"Table {i} Has Been Deleted ✔️")
    except Exception as Er:
        print(Er)    


Table music_Library Has Been Deleted ✔️
Table music_Library2 Has Been Deleted ✔️
Table music_Library3 Has Been Deleted ✔️


### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()